In [1]:
import _sqlite3
import pandas as pd
import numpy as np
import glob
from pathlib import Path

# The project paths
ROOT = Path("..") # This notebook is not in the root
RAW_DIR = ROOT / "data" / "raw"
PROCESSED_DIR = ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Paths to the raw data (CDC and AQI, respectively)
CDC_PATH = RAW_DIR / "CDC" / "cdc_raw.csv"
AQI_DIR = RAW_DIR / "AQI"

# Test that the paths are working expectedly
print("ROOT: ", ROOT.resolve())
print("CDC_PATH exists:", CDC_PATH.exists())
print("AQI_DIR exists:", AQI_DIR.exists())

ROOT:  C:\Users\Delanie\Documents\GitHub\CAP5771-Milestone-1
CDC_PATH exists: True
AQI_DIR exists: True


In [3]:
# Load and process the CDC data into "cdc_m1"
cdc_raw = pd.read_csv(CDC_PATH, low_memory=False)

# Create cdc_m1 table with only the CDC cols that we need
cdc_m1 = cdc_raw[[
    "Week Ending Date",
    "Geographic aggregation",
    "Total COVID-19 Admissions",
    "Total Influenza Admissions",
    "Total RSV Admissions"
]].copy()

# Rename the cols for consistency
cdc_m1 = cdc_m1.rename(columns={
    "Week Ending Date": "week",
    "Geographic aggregation": "state",
    "Total COVID-19 Admissions": "covid_admissions",
    "Total Influenza Admissions": "influenza_admissions",
    "Total RSV Admissions": "rsv_admissions"
})

# Make the week column dtype datetime and standardize the state column
cdc_m1["week"] = pd.to_datetime(cdc_m1["week"]).dt.normalize()
cdc_m1["state"] = cdc_m1["state"].astype(str).str.strip().str.upper()

# Make the admissions columns numeric
for c in ["covid_admissions", "influenza_admissions", "rsv_admissions"]:
    cdc_m1[c] = pd.to_numeric(cdc_m1[c], errors="coerce")

# Create a total_respiratory_admissions column by adding the three total admission columns together
cdc_m1["total_respiratory_admissions"] = (
    cdc_m1["covid_admissions"].fillna(0)
    + cdc_m1["influenza_admissions"].fillna(0)
    + cdc_m1["rsv_admissions"].fillna(0)
)

cdc_m1.head()

,week,state,covid_admissions,influenza_admissions,rsv_admissions,total_respiratory_admissions
0,2025-10-04,AK,5.0,2.0,1.0,8.0
1,2025-10-11,AK,4.0,0.0,0.0,4.0
2,2025-10-18,AK,5.0,0.0,1.0,6.0
3,2025-10-25,AK,2.0,1.0,0.0,3.0
4,2025-11-01,AK,10.0,0.0,3.0,13.0


In [14]:
# Load and process the AQI raw data

# Sort and gather the AQI files in the AQI directory
aqi_files = sorted(glob.glob(str(AQI_DIR / "*.csv")))
if not aqi_files:
    raise FileNotFoundError(f"No AQI data found in {AQI_DIR}")

# Concat the sorted AQI files
aqi_daily_raw = pd.concat((pd.read_csv(f) for f in aqi_files), ignore_index=True)

# Only gather the cols we need from the raw daily AQI data
aqi_daily = aqi_daily_raw[[
    "Date",
    "State Name",
    "AQI"
]].copy()

# Rename the columns for consistency
aqi_daily = aqi_daily.rename(columns={
    "Date": "date",
    "State Name": "state",
    "AQI": "aqi"
})

# Update the dtypes of date and aqi to datetime and numeric, respectively
aqi_daily["date"] = pd.to_datetime(aqi_daily["date"])
aqi_daily["aqi"] = pd.to_numeric(aqi_daily["aqi"], errors="coerce")

# Weekly ending Saturday, then normalize to date-only
aqi_daily["week"] = aqi_daily["date"].dt.to_period("W-SAT").apply(lambda r: r.end_time)
# Normalize the week times so they match the CDC weeks and can merge
aqi_daily["week"] = pd.to_datetime(aqi_daily["week"]).dt.normalize()

# Aggregate into weekly state-level AQI
aqi_m1 = (
    aqi_daily
    .groupby(["state", "week"], as_index=False)
    .agg(
        aqi_mean=("aqi", "mean"),
        aqi_p90=("aqi", lambda s: s.quantile(0.90)),
        aqi_max=("aqi", "max"),
        days_reported=("aqi", "count")
    )
)

# Examine the top five rows of aqi_m1
aqi_m1.head()

,state,week,aqi_mean,aqi_p90,aqi_max,days_reported
0,Alabama,2021-01-02,33.285714,53.0,55,21
1,Alabama,2021-01-09,42.246154,56.0,63,65
2,Alabama,2021-01-16,45.522388,61.0,72,67
3,Alabama,2021-01-23,39.500000,59.2,67,60
4,Alabama,2021-01-30,36.457627,56.6,67,59


In [21]:
# Robustly map the AQI state names to their abbreviations. This took a long time to debug and get done correctly.
US_STATE_ABBR = {
    "Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO",
    "Connecticut":"CT","Delaware":"DE","District of Columbia":"DC","Florida":"FL","Georgia":"GA",
    "Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY",
    "Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI","Minnesota":"MN",
    "Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV","New Hampshire":"NH",
    "New Jersey":"NJ","New Mexico":"NM","New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH",
    "Oklahoma":"OK","Oregon":"OR","Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC","South Dakota":"SD",
    "Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT","Virginia":"VA","Washington":"WA",
    "West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY",
    "Puerto Rico":"PR","Guam":"GU","Virgin Islands":"VI","American Samoa":"AS","Northern Mariana Islands":"MP"
}

def normalize_state_name(s: str) -> str:
    s = str(s).strip()
    # collapse multiple spaces
    s = " ".join(s.split())
    # common DC casing variants
    if s.lower() in {"district of columbia", "district of columbia ", "d.c.", "dc"}:
        return "District of Columbia"
    # title case for matching dict keys (handles 'new york' -> 'New York')
    return s.title()

# Normalize
aqi_m1["state_norm"] = aqi_m1["state_raw"].apply(normalize_state_name)

# If already 2-letter abbreviations, keep them; else map full names
aqi_m1["state_abbr"] = np.where(
    aqi_m1["state_norm"].str.len() == 2,
    aqi_m1["state_norm"].str.upper(),
    aqi_m1["state_norm"].map(US_STATE_ABBR)
)

# Drop any rows of data outside the U.S.
aqi_m1 = aqi_m1[aqi_m1["state_abbr"].notna()].copy()

# Diagnostics
unmapped = aqi_m1.loc[aqi_m1["state_abbr"].isna(), "state_raw"].dropna().unique()
print("Unmapped count:", len(unmapped))
print("First 20 unmapped examples:", unmapped[:20])
print("Example mapped:", aqi_m1["state_abbr"].dropna().unique()[:10])

# Update the aqi_m1 state column after the states have been properly mapped to their abbreviations
aqi_m1["state"] = aqi_m1["state_abbr"]

Unmapped count: 0
First 20 unmapped examples: <StringArray>
[]
Length: 0, dtype: str
Example mapped: <StringArray>
['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL']
Length: 10, dtype: str


In [22]:
# Compatibility checks before we merge the datasets
print("CDC weeks:", cdc_m1["week"].min(), "to", cdc_m1["week"].max())
print("AQI weeks:", aqi_m1["week"].min(), "to", aqi_m1["week"].max())

state_overlap = set(cdc_m1["state"]).intersection(set(aqi_m1["state"]))
week_overlap = set(cdc_m1["week"]).intersection(set(aqi_m1["week"]))
print("Overlapping states:", len(state_overlap))
print("Overlapping weeks:", len(week_overlap))

CDC weeks: 2020-08-08 00:00:00 to 2026-01-31 00:00:00
AQI weeks: 2021-01-02 00:00:00 to 2025-11-15 00:00:00
Overlapping states: 53
Overlapping weeks: 255


In [28]:
# Merge the datasets... finally
merged_m1 = pd.merge(
    cdc_m1,
    aqi_m1,
    on=["state", "week"],
    how="inner"
)

# Get rid of the helper columns
merged_m1 = merged_m1[[
    "state",
    "week",
    "covid_admissions",
    "influenza_admissions",
    "rsv_admissions",
    "total_respiratory_admissions",
    "aqi_mean",
    "aqi_p90",
    "aqi_max",
    "days_reported"
]].copy()

print(merged_m1.shape)
merged_m1.head()

(13080, 10)


,state,week,covid_admissions,influenza_admissions,rsv_admissions,total_respiratory_admissions,aqi_mean,aqi_p90,aqi_max,days_reported
0,AK,2025-10-04,5.0,2.0,1.0,8.0,20.000000,24.4,26,3
1,AR,2021-01-02,NaN,20.0,NaN,20.0,21.705882,31.2,35,17
2,AR,2021-01-09,NaN,20.0,NaN,20.0,34.078431,42.0,50,51
3,AR,2021-01-16,NaN,17.0,NaN,17.0,42.035088,57.0,73,57
4,AR,2021-01-23,NaN,13.0,NaN,13.0,35.115385,46.9,55,52


In [30]:
# Save all three of the processed files to the processed data folder

cdc_m1.to_csv(PROCESSED_DIR / "cdc_m1.csv", index=False)
aqi_m1.to_csv(PROCESSED_DIR / "aqi_m1.csv", index=False)
merged_m1.to_csv(PROCESSED_DIR / "merged_m1.csv", index=False)

In [31]:
# Create the DB file!!!
db_path = PROCESSED_DIR / "milestone1.db"
conn = _sqlite3.connect(db_path)

cdc_m1.to_sql("cdc_m1", conn, index=False, if_exists="replace")
aqi_m1.to_sql("aqi_m1", conn, index=False, if_exists="replace")
merged_m1.to_sql("merged_m1", conn, index=False, if_exists="replace")

conn.close()